#### Setup

In [2]:
import sys
import numpy as np
import pandas as pd
import math
import random
from concorde.tsp import TSPSolver
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
print(sys.version)

3.7.6 | packaged by conda-forge | (default, Mar 23 2020, 22:45:16) 
[Clang 9.0.1 ]


In [3]:
# GLOBAL VARIABLES
field_width = 100
field_height = 100
depot_x = 50
depot_y = 50

In [4]:
class Instance():
    
    def __init__(self, xlocs, ylocs, demands, capacity, route_size, overlap_size):

        self.num_customers = len(demands)-1
        self.fleet_size = len(demands)/route_size
        self.overlap_size = overlap_size
        self.capacity = capacity
        self.demands = demands
        self.xlocs = xlocs
        self.ylocs = ylocs
        self.route_size = route_size
        self.distances = self.calc_distance_matrix()
        
    def calc_distance_matrix(self):

        distances = np.zeros((self.num_customers+1, self.num_customers+1), dtype=float)
        for i in range(self.num_customers+1):
            for j in range(self.num_customers+1):
                new_dist = math.sqrt((self.xlocs[i]-self.xlocs[j])**2 + (self.ylocs[i]-self.ylocs[j])**2)
                distances[i,j] = new_dist
        return distances

    def get_lowerbound(self):
        return (2/self.capacity) * sum([self.demands[i]*self.distances[0,i]
                                        for i in range(len(self.demands))])

In [5]:
def solve_TSP(inst):
    
    solver = TSPSolver.from_data(inst.xlocs, inst.ylocs, 'EUC_2D')
    solution = solver.solve()
    return solution


def get_dedicated_routes(inst,tour):
    
    tour =  tour[1:]
    routes = []
    for i in range(0,len(tour),inst.route_size):
        new_route = tour[i:i+inst.route_size]
        routes.append(new_route)
    return routes


In [7]:
def get_circular_cost(inst,segment):
    return sum([inst.distances[segment[i],segment[i+1]] for i in range(len(segment)-1)])

def get_radial_cost(inst,segment):
    """Assumes vehicle travels to/from the depot at segment endpoints."""
    return inst.distances[0,segment[0]] + inst.distances[0,segment[-1]]

In [6]:
N = 3
dmin = 0
dmax = 8
cust_x = field_width*np.random.random(N)
cust_y = field_height*np.random.random(N)
cust_dems = np.random.randint(dmin,dmax,N)
xlocs = np.append([depot_x], cust_x)
ylocs = np.append([depot_y], cust_y)
demands = np.append([0], cust_dems)
inst = Instance(xlocs, ylocs, demands, capacity=20, route_size=5, overlap_size=5)

In [8]:
tsp = solve_TSP(inst)
routes = get_dedicated_routes(inst, tsp.tour)
print('Big tour:', tsp.tour)
print('Routes:', routes)
print('Radial costs:', [get_radial_cost(inst,seg).round(1) for seg in routes])
print('Circular costs:', [get_circular_cost(inst,seg).round(1) for seg in routes])

Big tour: [0 3 1 2]
Routes: [array([3, 1, 2], dtype=int32)]
Radial costs: [96.8]
Circular costs: [98.6]
